### Setup Project

In [1]:
import os

os.environ["CUDA_VERSION"] = ""
from src import create_and_set_project

In [2]:
project = create_and_set_project(
    name="odsc-west-2023",
    source="git://github.com/igz-us-sales/k3d-mlrun#master",
    # source="v3io:///bigdata/odsc-west-2023.zip",
    secrets_file="secrets.env",
)

> 2023-10-29 22:35:16,465 [info] Project loaded successfully: {'project_name': 'odsc-west-2023'}


### Run Training Pipeline

In [3]:
run_id = project.run(
    name="main",
    arguments={
        "dataset_name" : "databricks/databricks-dolly-15k",
        "dataset_text_field" : "text",
        "urls_file" : "./data/mlops_blogs.txt",
        "pretrained_tokenizer" : "meta-llama/Llama-2-7b-hf",
        "pretrained_model" : "meta-llama/Llama-2-7b-hf",
        "regular_model_name" : "regular-adapter",
        "pirate_model_name" : "pirate-adapter",
        "max_steps" : 1000,
        "logging_steps" : 100,
        "save_steps" : 500
    },
    dirty=True,
    watch=False,
)

> 2023-10-29 22:35:22,621 [warning] WARNING!, you seem to have uncommitted git changes, use .push()
> 2023-10-29 22:35:23,277 [info] submitted pipeline odsc-west-2023-main 2023-10-29 22-35-23 id=f6ef0d0b-375d-4a6b-acce-143a2d0142e5
> 2023-10-29 22:35:23,278 [info] Pipeline run id=f6ef0d0b-375d-4a6b-acce-143a2d0142e5, check UI for progress


> 2023-10-29 22:35:23,769 [info] started run workflow odsc-west-2023-main with run id = 'f6ef0d0b-375d-4a6b-acce-143a2d0142e5' by kfp engine


### Deploy Serving Function

In [ ]:
serving_fn = project.get_function("serving")
serving_fn.set_env("CUDA_VERSION", "")
serving_fn.with_node_selection(
    node_selector={"app.iguazio.com/node-group" : "added-v100"}
)
serving_fn.with_limits(gpus=1, patch=True)
serving_fn.spec.readiness_timeout = 3000
serving_fn.spec.min_replicas = 1
serving_fn.spec.max_replicas = 1

model_name = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
graph = serving_fn.set_topology("flow", engine="async")
graph.to(handler="preprocess", name="src.functions.serving.preprocess") \
     .to("src.functions.serving.LLMModelServer",
         name="llm",
         model_args={
             "load_in_4bit": True,
             "device_map": "auto",
             "trust_remote_code": True,
             "return_dict" : True
         },
         tokenizer_name=model_name,
         model_name=model_name,
         adapters={
             "pirate" : project.get_artifact_uri("pirate-adapter"),
             "regular" : project.get_artifact_uri("regular-adapter")
         },
         stop_token = "##"
        ).respond()

serving_fn.plot(rankdir='LR')

In [ ]:
project.deploy_function(serving_fn)

### Test Serving

In [3]:
serving_fn = project.get_function("serving")

In [13]:
%%time
prompt = "can mlrun work with spark?"

body = {
    "prompt" : prompt,
    "adapter" : "pirate",
    "rag" : True,
    "k" : 2,
    "max_new_tokens": 50,
    "temperature": 0.5,
    "top_p": 0.7,
}

response = serving_fn.invoke(path='/predict', body=body)

> 2023-10-29 21:19:23,245 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-odsc-west-2023-serving.default-tenant.svc.cluster.local:8080/predict'}
CPU times: user 15.5 ms, sys: 1.14 ms, total: 16.6 ms
Wall time: 6.84 s


In [14]:
print(response["outputs"]["prediction"])

Aye, mlrun can work with spark. Spark be a popular open-source engine for building scalable and reliable machine learning systems, and it can be used in conjunction with mlrun to manage and monitor th' spark clusters


### Chatbot UI

In [9]:
import json

import gradio as gr
import requests

In [10]:
serving_fn = project.get_function("serving")

In [11]:
def generate(prompt, adapter, rag, k, temperature, max_length, top_p, top_k, repetition_penalty):
    # Build the request for our serving graph:
    inputs = {
        "prompt": prompt,
        "adapter" : adapter.lower(),
        "rag" : rag,
        "k" : k,
        "temperature": temperature,
        "max_length": max_length,
        "top_p": top_p,
        "top_k": top_k,
        "repetition_penalty": repetition_penalty,
    }
    
    output = serving_fn.invoke(path='/predict', body=inputs)["outputs"]["prediction"]
    
    if not output:
        output = "Context window exceeded - increase max length"
    
    # Return the response:
    return output


# Set up a Gradio frontend application:
with gr.Blocks(analytics_enabled=False, theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """# LLM Playground
Play with the `generate` configurations and see how they make the LLM's responses better or worse.
"""
    )
    with gr.Row():
        with gr.Column(scale=5):
            with gr.Row():
                chatbot = gr.Chatbot()
            with gr.Row():
                prompt = gr.Textbox(label="Subject to ask about:", placeholder="Type a question and Enter")

        with gr.Column(scale=1):
            adapter = gr.Dropdown(label="Adapter", value="Pirate", choices=["Regular", "Pirate"], info="Choose fine tuned adapter")
            rag = gr.Checkbox(label="RAG Enrichment", value=True)
            k = gr.Slider(minimum=0, maximum=5, value=2, label="Num Sources", step=1)
            temperature = gr.Slider(minimum=0, maximum=1, value=0.9, label="Temperature", info="Choose between 0 and 1")
            max_length = gr.Slider(minimum=0, maximum=1500, value=150, label="Maximum length", info="Choose between 0 and 1500")
            top_p = gr.Slider(minimum=0, maximum=1, value=0.5, label="Top P", info="Choose between 0 and 1")
            top_k = gr.Slider(minimum=0, maximum=500, value=25, label="Top k", info="Choose between 0 and 500")
            repetition_penalty = gr.Slider(minimum=0, maximum=1, value=1, label="repetition penalty", info="Choose between 0 and 1")
            clear = gr.Button("Clear")

    def respond(prompt, chat_history, adapter, rag, k, temperature, max_length, top_p, top_k, repetition_penalty):
        bot_message = generate(prompt, adapter, rag, k, temperature, max_length, top_p, top_k, repetition_penalty)
        chat_history.append((prompt, bot_message))

        return "", chat_history

    prompt.submit(respond, [prompt, chatbot, adapter, rag, k, temperature, max_length, top_p, top_k, repetition_penalty], [prompt, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

In [ ]:
demo.launch(share=True, height=1500)

In [29]:
# demo.close()